# Homework 0 Content Analysis

## YIMIN LI liym15@uchicago.edu

# Week 0 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [91]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas as pd#gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [24]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [25]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [26]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:2000])


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"Xo9QHwpAAD8AAHnHtJ0AAADO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":946472270,"wgRevisionId":946472270,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology a

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [27]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [28]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:1000])






Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"Xo9QHwpAAD8AAHnHtJ0AAADO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":946472270,"wgRevisionId":946472270,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology articles needing expert attention","Media articles needing expert attention","Articles prone to spam from October 

This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [29]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [30]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is a research method for studying documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in conten

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [31]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is a research method for stud...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   Quantitative content analysis highlights frequ...
7   Siegfried Kracauer provides a critique of quan...
8   More generally, content analysis is research u...
9   By having contents of communication available ...
10  Computer-assisted analysis can help with large...
11  Robert Weber notes: "To make valid inferences ...
12  There are five types of texts in content analy...
13  Over the years, content analysis has been appl...
14  In recent times, particularly with the advent ...
15  Quantitative content analysis has enjoyed a re...
16  Content analysis can also be described as stud...
17  Manifest content is read

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [32]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [33]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [34]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [37]:
wiki_url_china = "https://en.wikipedia.org/wiki/Politics_of_China"
requests.get(wiki_url_china)

<Response [200]>

In [90]:
content_china = requests.get(wiki_url_china)
soup = bs4.BeautifulSoup(content_china.text, 'html.parser')
soup_content = soup.findAll('p')

ls_paragraph = []
for soup_paragraph in soup_content:
    temp_text = soup_paragraph.text
    temp_text = re.sub(r'\[.*?]', '', temp_text) #remove "[]"
    temp_text = re.sub(r'\n', '', temp_text) # remove "\n"
    temp_text = re.sub(r'.*[^.]$', '', temp_text) #remove text not ending in period "."
    if temp_text:
        print(temp_text)
        print("\n")
        ls_paragraph.append(temp_text)

The politics of the People's Republic of China takes place in a framework of a  socialist republic run by a single party, the Communist Party of China, headed by the General Secretary. State power within the People's Republic of China (PRC) is exercised through the Communist Party, the Central People's Government (State Council) and their provincial and local representation. The state uses Internal Reference , secret documents produced by Xinhua News Agency, similar to US's President's Daily Brief, though delivered to most of its officials according to level of secrecy of the information, a major source of information of the society.


Each local Bureau or office is under the coequal authority of the local leader and the leader of the corresponding office, bureau or ministry at the next higher level. People's Congress members at the county level are elected by voters. These county-level People's Congresses have the responsibility of oversight of local government and elect members to th

In [93]:
df_para = pd.DataFrame({'paragraph-text': ls_paragraph})
print(df_para)

                                       paragraph-text
0   The politics of the People's Republic of China...
1   Each local Bureau or office is under the coequ...
2   The President of China is the head of state, s...
3   China's population, geographical vastness, and...
4   Central government leaders must, in practice, ...
5   The social, cultural, and political as well as...
6   The PRC consists of 21 provinces (excluding th...
7   The Communist Party of China calls China's sys...
8   According to a China Today editorial, "Consult...
9   In 2012, Li Changjian, a member of the Nationa...
10  However, elections are also an element in soci...
11  The 90 million-member Communist Party of China...
12  Nevertheless, in all governmental institutions...
13  Constitutionally, the party's highest body is ...
14  The primary organs of state power are the Nati...
15  At the same time, there has been a convention ...
16  Under the Constitution of China, the NPC is th...
17  Currently, local governm

In [128]:
# Filter sentences related to China and construct a sentence dataframe
ls_sent_China = []
for para in ls_paragraph:
    ls_sentence = re.split(r'\.|\!|\?', para)
    for sent in ls_sentence[:-1]:
        searchObj = re.search(r'China|Chinese|PRC|Beijing', para)
        if searchObj:
            sent = sent.strip()
            if sent:
                ls_sent_China.append(sent)
print(len(ls_sent_China))
ls_sent_China

164


["The politics of the People's Republic of China takes place in a framework of a  socialist republic run by a single party, the Communist Party of China, headed by the General Secretary",
 "State power within the People's Republic of China (PRC) is exercised through the Communist Party, the Central People's Government (State Council) and their provincial and local representation",
 "The state uses Internal Reference\xa0, secret documents produced by Xinhua News Agency, similar to US's President's Daily Brief, though delivered to most of its officials according to level of secrecy of the information, a major source of information of the society",
 'Each local Bureau or office is under the coequal authority of the local leader and the leader of the corresponding office, bureau or ministry at the next higher level',
 "People's Congress members at the county level are elected by voters",
 "These county-level People's Congresses have the responsibility of oversight of local government and e

In [129]:
df_sent = pd.DataFrame({'sentence-text': ls_sent_China})
df_sent.head(10)

,sentence-text
0,The politics of the People's Republic of China...
1,State power within the People's Republic of Ch...
2,"The state uses Internal Reference , secret doc..."
3,Each local Bureau or office is under the coequ...
4,People's Congress members at the county level ...
5,These county-level People's Congresses have th...
6,"The Provincial People's Congress, in turn, ele..."
7,The ruling Communist Party committee at each l...
8,"The President of China is the head of state, s..."
9,The Premier of China is the head of government...



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [130]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


In [131]:
print(contentPTags)

[<p><b>Content analysis</b> is a research method for studying <a href="/wiki/Document" title="Document">documents</a> and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.
</p>, <p>Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of <a href="/wiki/Text_(literary_theory)" title="Text (literary theory)">texts</a> or artifacts which are <a href="/wiki/Coding_(social_sciences)" title="Coding (social sciences)">assigned labels (sometimes called codes)</a> to indicate the presence of interesting, <a hr

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [132]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,6
7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,8
9,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [133]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [134]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [236]:
base_url = 'https://en.wikipedia.org/wiki/Politics_of_China'
wiki_base_url = 'https://en.wikipedia.org'

In [237]:
content = requests.get(base_url)
soup = bs4.BeautifulSoup(content.text, 'html.parser')
soup_content = soup.findAll('p')
for paragraphNum, pTag in enumerate(soup_content):
    links = pTag.findAll('a', href=re.compile('/wiki/'))
links[0]

<a href="/wiki/African_Development_Bank" title="African Development Bank">AfDB</a>

In [253]:
# Use recursion to find all links: We only find links that are two-link deep 
# and only scrape first five links

def getURLs(url, ls_url, count):
    if len(ls_url) >= 30:
        return ls_url
    content = requests.get(url)
    soup = bs4.BeautifulSoup(content.text, 'html.parser')
    soup_content = soup.body.findAll('p')

    for paragraphNum, pTag in enumerate(soup_content):
        links = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
        if len(links) == 0:
            continue
        else: 
            for link in links[:5]:
                link_url = link.get('href')
                final_url = urllib.parse.urljoin(wiki_base_url, link_url)
                link_text = link.text
                count += 1 # We use this to calculate how much deep we go
                if count <= 2: 
                    ls_url = getURLs(final_url, ls_url, count)
                ls_url.append((final_url, paragraphNum, linkText))

    return ls_url

In [254]:
all_urls = getURLs(base_url, [], 0)
all_urls

[('https://en.wikipedia.org/wiki/International_law', 1, 'paradigm'),
 ('https://en.wikipedia.org/wiki/Centralized_government', 1, 'paradigm'),
 ('https://en.wikipedia.org/wiki/Sovereignty', 1, 'paradigm'),
 ('https://en.wikipedia.org/wiki/State_(polity)', 1, 'paradigm'),
 ('https://en.wikipedia.org/wiki/Government', 1, 'paradigm'),
 ('https://en.wikipedia.org/wiki/Diplomatic_recognition', 2, 'paradigm'),
 ('https://en.wikipedia.org/wiki/List_of_states_with_limited_recognition',
  2,
  'paradigm'),
 ('https://en.wikipedia.org/wiki/Diplomacy', 2, 'paradigm'),
 ('https://en.wikipedia.org/wiki/Nation_state', 3, 'paradigm'),
 ('https://en.wikipedia.org/wiki/Multinational_corporation', 3, 'paradigm'),
 ('https://en.wikipedia.org/wiki/Peace_of_Westphalia', 3, 'paradigm'),
 ('https://en.wikipedia.org/wiki/H._V._Evatt', 4, 'paradigm'),
 ('https://en.wikipedia.org/wiki/High_Court_of_Australia', 4, 'paradigm'),
 ('https://en.wikipedia.org/wiki/Self-determination', 5, 'paradigm'),
 ('https://en.wi

In [256]:
len(all_urls)

297

In [259]:
df_all_links = pd.DataFrame(all_urls, columns=['source', 'num of paragraph', 'text'])
df_all_links.head()

,source,num of paragraph,text
0,https://en.wikipedia.org/wiki/International_law,1,paradigm
1,https://en.wikipedia.org/wiki/Centralized_gove...,1,paradigm
2,https://en.wikipedia.org/wiki/Sovereignty,1,paradigm
3,https://en.wikipedia.org/wiki/State_(polity),1,paradigm
4,https://en.wikipedia.org/wiki/Government,1,paradigm


In [262]:
df_para['num of paragraph'] = df_para.index
df_para.head()

,paragraph-text,num of paragraph
0,The politics of the People's Republic of China...,0
1,Each local Bureau or office is under the coequ...,1
2,"The President of China is the head of state, s...",2
3,"China's population, geographical vastness, and...",3
4,"Central government leaders must, in practice, ...",4


In [267]:
df_merge = pd.merge(df_all_links, df_para, on='num of paragraph')
print(df_merge.shape)
df_merge.head()

(271, 4)


,source,num of paragraph,text,paragraph-text
0,https://en.wikipedia.org/wiki/International_law,1,paradigm,Each local Bureau or office is under the coequ...
1,https://en.wikipedia.org/wiki/Centralized_gove...,1,paradigm,Each local Bureau or office is under the coequ...
2,https://en.wikipedia.org/wiki/Sovereignty,1,paradigm,Each local Bureau or office is under the coequ...
3,https://en.wikipedia.org/wiki/State_(polity),1,paradigm,Each local Bureau or office is under the coequ...
4,https://en.wikipedia.org/wiki/Government,1,paradigm,Each local Bureau or office is under the coequ...


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [268]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3599,"posts-type":false,"posts":[{"id":"615154382228586496","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/615154382228586496","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/615154382228586496\/dont-sweat-it","type":"photo","date-gmt":"2020-04-12 02:02:14 GMT","date":"Sun, 12 Apr 2020 04:02:14","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1586656934,"format":"html","reblog-key":"vUuqLQou","slug":"dont-sweat-it","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"615154382228586496\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_615154382228586496\"><iframe id=\"like_iframe_615154382228586496\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762b8d18f9d06#name=lolcats-lol-ca

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [270]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [271]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [273]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,615154382228586496,https://66.media.tumblr.com/0d8f679e85ab310a97...,"Sun, 12 Apr 2020 04:02:14","[cat, cats, lol, lolcat, lolcats]",png
1,615003293374119936,https://66.media.tumblr.com/d05e62cfa8fab347ba...,"Fri, 10 Apr 2020 12:00:45","[cat, cats, lol, lolcat, lolcats]",jpg
2,614995720352710657,https://66.media.tumblr.com/a6e290b8471e09589f...,"Fri, 10 Apr 2020 10:00:23","[cat, cats, lol, lolcat, lolcats]",jpg
3,614723926577348608,https://66.media.tumblr.com/7292003ce717f2cecc...,"Tue, 07 Apr 2020 10:00:20","[cat, cats, lol, lolcat, lolcats]",jpg
4,614655994554941440,https://66.media.tumblr.com/90e5c1ab9abbe0ba00...,"Mon, 06 Apr 2020 16:00:35","[cat, cats, lol, lolcat, lolcats]",jpg
5,614648463725133824,https://66.media.tumblr.com/7ae866fa4717c2441c...,"Mon, 06 Apr 2020 14:00:53","[cat, cats, lol, lolcat, lolcats]",jpg
6,614640908693127168,https://66.media.tumblr.com/8e2642f3832f424e22...,"Mon, 06 Apr 2020 12:00:48","[cat, cats, lol, lolcat, lolcats]",jpg
7,614633334778281984,https://66.media.tumblr.com/935677e4f4444055ae...,"Mon, 06 Apr 2020 10:00:25","[cat, cats, lol, lolcat, lolcats]",jpg
8,614346537985785856,https://66.media.tumblr.com/4fd0402ebbe6d77e23...,"Fri, 03 Apr 2020 06:01:54","[cat, cats, lol, lolcat, lolcats]",jpg
9,614172929162346496,https://66.media.tumblr.com/f3edc9b70ea8f53592...,"Wed, 01 Apr 2020 08:02:28","[cat, cats, lol, lolcat, lolcats]",png


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [274]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [275]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [278]:
with open('data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [279]:
targetDir = 'data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [283]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
alls_well_that_ends_well.txt,"All's Well, that Ends Well\n\nActus primus. Sc..."
anthonie_and_cleopatra.txt,"The Tragedie of Anthonie, and Cleopatra\n\nAct..."
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
comedy_of_errors.txt,"DRAMATIS PERSONAE\n\nSOLINUS, Duke of Ephesus\..."
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...
cymbeline.txt,The Tragedie of Cymbeline\n\nActus Primus. Sco...
hamlet.txt,The Tragedie of Hamlet\n\nActus Primus. Scoena...
julius_caesar.txt,"Dramatis Personae\n\n JULIUS CAESAR, Roman st..."
king_henry_4_p1.txt,The First Part of Henry the Fourth\n\nwith the...
king_henry_4_p2.txt,"KING HENRY IV, SECOND PART\n\nby William Shake..."


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [284]:
information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.5
%����
97 0 obj
<<
/Length 2988      
/Filter /FlateDecode
>>
stream
xڝYY��~�_��T,U�����f<{����z\[)o(
���!�������n��4�J�/b�� }7�9�s������o���I�,�|�a�d�J��M��y�9?�~<i]כHE���	�}����e��k��esp�ߋ�շyS�Jf��|�?>���׺�+��y7����==w��8�����M<�x�
߉�M�Q�k�k�tW�C�6덂
������拍���Y��o乤�8s���Fy-�����g��vGvM ����3���}���[+o�y��שnٰ|7N6UŽ�f����l����D�D��|[i�n��MY��f�3��W�n`�>���X!~lڧ��>��K�7�
��P�z������s;
�zx�����t��|ӎ���n���V{�Av@�Wt:�!bc!*���>����΋�@R�Q`��_�A���Aл�yG+J���z[��\[G�1-r��U�m�A�|��'�2


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [285]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [286]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [287]:
print(readPDF(infoExtractionBytes)[:550])

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determini


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [288]:
example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [289]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [290]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [291]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [311]:
# text
with open('data/Shakespeare/as_you_like_it.txt') as f:
    txt = f.read()
print(txt[:5000])

AS YOU LIKE IT

by William Shakespeare



DRAMATIS PERSONAE.


  DUKE, living in exile
  FREDERICK, his brother, and usurper of his dominions
  AMIENS, lord attending on the banished Duke
  JAQUES,   "      "       "  "     "      "
  LE BEAU, a courtier attending upon Frederick
  CHARLES, wrestler to Frederick
  OLIVER, son of Sir Rowland de Boys
  JAQUES,   "   "  "    "     "  "
  ORLANDO,  "   "  "    "     "  "
  ADAM,   servant to Oliver
  DENNIS,     "     "   "
  TOUCHSTONE, the court jester
  SIR OLIVER MARTEXT, a vicar
  CORIN,    shepherd
  SILVIUS,     "
  WILLIAM, a country fellow, in love with Audrey
  A person representing HYMEN
  ROSALIND, daughter to the banished Duke
  CELIA, daughter to Frederick
  PHEBE, a shepherdes
  AUDREY, a country wench
  Lords, Pages, Foresters, and Attendants


<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM
SHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS
PROVIDED BY PROJECT GUTENBERG ETEXT OF CARNEGIE MELLON UNI

In [321]:
# Process the input string and output a dataframe
def text_to_df(s):
    text_split = s[:5000].split('\n')
    final_split = []
    for text in text_split:
        text = re.sub(r"[^A-Za-z0-9,.' ]", '', text)
        text = text.strip()
        if text:
            final_split.append(text)

    df_text = pd.DataFrame(final_split, columns=['text_line'])
    return df_text
    #df_text.tail(20)

In [322]:
df_txt = text_to_df(txt[:5000])
df_txt.tail(20)

,text_line
77,"away my blood, were there twenty brothers betw..."
78,"much of my father in me as you, albeit I confe..."
79,before me is nearer to his reverence.
80,"OLIVER. What, boy ..."
81,"ORLANDO. Come, come, elder brother, you are to..."
82,"OLIVER. Wilt thou lay hands on me, villain"
83,ORLANDO. I am no villain I am the youngest son...
84,Boys. He was my father and he is thrice a vill...
85,a father begot villains. Wert thou not my brot...
86,take this hand from thy throat till this other...


In [339]:
# pdf
# To read the pdf, we borrowed the idea in the instructions.
information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'
infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)
str_pdf = readPDF(infoExtractionBytes)[:2000]
print(str_pdf)

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determining the relationship (if any) between airline an-
nouncements of fare increases and the behavior of their stocks the next day. His-
torical data about stock prices is easy to come by, but what about the airline an-
nouncements? You will need to know at least the name of the airline, the nature of
the proposed fare hike, the dates of the announcement, and possibly the response of
other airlines. Fortunately, these can be all found in news articles 

In [343]:
df_pdf = text_to_df(str_pdf)
df_pdf.head(10)

,text_line
0,Speech and Language Processing. Daniel Jurafsk...
1,rights reserved.
2,"Draft of November 7, 2016."
3,Copyright ccid13 2016.
4,All
5,CHAPTER
6,21 Information Extraction
7,"I am the very model of a modern MajorGeneral,"
8,"Ive information vegetable, animal, and mineral,"
9,"I know the kings of England, and I quote the g..."


In [344]:
# doc
# To read the doc, we borrowed the idea in the instructions.
example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'
str_doc=''

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    str_doc += paragraph.text
    str_doc += "\n"
print(str_doc)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

In [345]:
df_doc = text_to_df(str_doc)
df_doc.head(5)

,text_line
0,Week 1 Retreiving and Preparing Text for Mach...
1,"This week, we begin by begging, borrowing and ..."
2,For this notebook we will be using the followi...
3,All these packages need to be installed from pip
4,import requests for http requests


### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

